In [144]:
import pandas as pd
import numpy as np

In [251]:
slots = pd.read_csv('DataFiles/NCAATourneySlots.csv')
seeds = pd.read_csv('DataFiles/NCAATourneySeeds.csv',index_col=['Season','Seed'])

In [476]:
tourney = pd.read_csv('DataFiles/NCAATourneyCompactResults.csv')

In [408]:
predict = pd.read_csv('submission_2019-3-10_20:47.csv')
predict['Season'] = predict['ID'].str.split('_').apply(lambda x: x[0]).astype(int)
predict['TeamA'] = predict['ID'].str.split('_').apply(lambda x: x[1]).astype(int)
predict['TeamB'] = predict['ID'].str.split('_').apply(lambda x: x[2]).astype(int)
predict = predict.set_index(['Season','TeamA','TeamB'])
predict = predict.drop(columns='ID')

In [542]:
teams = pd.read_csv('DataFiles/Teams.csv',index_col='TeamID',usecols=(0,1))
max_length = teams['TeamName'].str.len().max()
teams['TeamName'] = teams['TeamName'].apply(lambda x: x + ' '*(max_length - len(x)))

In [509]:
seasons = pd.read_csv('DataFiles/Seasons.csv',index_col='Season')

In [569]:
#######################
results = slots.copy()
results = results[results['Season'] >= 2014]
results['Round'] = results['Slot'].str.slice(stop=2)
results['Region'] = results['Slot'].str.extract('([WXYZ])')
results['TrueSS'] = np.nan
results['TrueWS'] = np.nan
first4 = False
for r in ['W1','X1','Y1','Z1']+['R%d' % (n+1) for n in range(6)]:
    for i,game in results[results['Round'] == r].iterrows():
        teamSS = game['TrueSS']
        teamWS = game['TrueWS']
        if np.isnan(teamSS):
            teamSS = seeds.loc[game['Season'],game['StrongSeed']]['TeamID']
            mySS = teamSS*1
            results.at[i,'TrueSS'] = teamSS
            results.at[i,'MySS'] = mySS
            results.at[i,'WFixed'] = 1
        else:
            results.at[i,'WFixed'] = 0
            mySS = game['MySS']
            
        if np.isnan(teamWS):
            teamWS = seeds.loc[game['Season'],game['WeakSeed']]['TeamID']
            myWS = teamWS*1
            results.at[i,'TrueWS'] = teamWS
            results.at[i,'MyWS'] = myWS
            results.at[i,'LFixed'] = 1
        else:
            results.at[i,'LFixed'] = 0
            myWS = game['MyWS']

        low = min(mySS,myWS)
        high = max(mySS,myWS)
        
        #print(game['Season'],game['Slot'],game['StrongSeed'],game['WeakSeed'],teamSS,teamWS,mySS,myWS,r,i)

        prob = predict.loc[game['Season'],low,high].values[0]
        if low == myWS:
            prob = 1-prob
        results.at[i,'Prob'] = prob
        
        match = tourney[(((tourney['WTeamID'] == teamSS) & (tourney['LTeamID'] == teamWS)) | \
                        ((tourney['WTeamID'] == teamWS) & (tourney['LTeamID'] == teamSS))) & \
                        (tourney['Season'] == game['Season'])]
        
        if match['WTeamID'].values[0] == teamSS:
            results.at[i,'Result'] = 1 
            results.at[i,'ScoreSS'] = match['WScore'].values[0]
            results.at[i,'ScoreWS'] = match['LScore'].values[0]
        else:
            results.at[i,'Result'] = 0
            results.at[i,'ScoreSS'] = match['LScore'].values[0]
            results.at[i,'ScoreWS'] = match['WScore'].values[0]
        
        if r != 'R6':
            idx = results[((results['StrongSeed'] == game['Slot']) | (results['WeakSeed'] == game['Slot'])) & \
            (results['Season'] == game['Season'])].index[0]
            
            if prob > 0.5:
                win = mySS
            else:
                win = myWS
            
            if results.loc[idx,'StrongSeed'] == game['Slot']:
                results.at[idx,'TrueSS'] = match['WTeamID']
                if game['StrongSeed'][-1] == 'a' and not first4:
                    results.at[idx,'MySS'] = match['WTeamID']
                else:
                    results.at[idx,'MySS'] = win
            else:
                results.at[idx,'TrueWS'] = match['WTeamID']
                if game['WeakSeed'][-1] == 'b' and not first4:
                    results.at[idx,'MyWS'] = match['WTeamID']
                else:
                    results.at[idx,'MyWS'] = win
                            
results[['TrueSS','TrueWS','Result','ScoreSS','ScoreWS']] = \
    results[['TrueSS','TrueWS','Result','ScoreSS','ScoreWS']].astype(int)

results['CorrectPred'] = results.apply(lambda row: 1 if (round(row['Prob']) == row['Result'] and \
                                row['TrueSS'] == row['MySS'] and \
                                row['TrueWS'] == row['MyWS']) else 0,axis=1)
results['PointsPossible'] = results['Round'].apply(lambda r: 2**(int(r[1])-1) if r[0] == 'R' else 0)
results['PointsRewarded'] = results['CorrectPred']*results['PointsPossible']

In [570]:
results.head()

,Season,Slot,StrongSeed,WeakSeed,Round,Region,TrueSS,TrueWS,MySS,WFixed,MyWS,LFixed,Prob,Result,ScoreSS,ScoreWS,CorrectPred,PointsPossible,PointsRewarded
1849,2014,R1W1,W01,W16,R1,W,1438,1157,1438.0,1.0,1157.0,1.0,0.877464,1,70,59,1,1,1
1850,2014,R1W2,W02,W15,R1,W,1437,1454,1437.0,1.0,1454.0,1.0,0.914672,1,73,53,1,1,1
1851,2014,R1W3,W03,W14,R1,W,1235,1300,1235.0,1.0,1300.0,1.0,0.962412,1,93,75,1,1,1
1852,2014,R1W4,W04,W13,R1,W,1277,1174,1277.0,1.0,1174.0,1.0,0.839033,1,93,78,1,1,1
1853,2014,R1W5,W05,W12,R1,W,1153,1217,1153.0,1.0,1217.0,1.0,0.660997,0,57,61,0,1,0


In [568]:
s = 2018
seeds_rev = seeds.reset_index().set_index(['Season','TeamID'])
for reg in list('WXYZ'):
    regName = seasons.loc[s,'Region%s' % reg]
    for rnd in ['R%d' % (n+1) for n in range(4)]:
        print('%s: Round %s' % (regName,rnd[1]))
        subset = results[(results['Round'] == rnd) & (results['Season'] == s) & (results['Region'] == reg)]
        for i,game in subset.iterrows():
            print('\t%s\t%s %d\t%s %0.2f' % (seeds_rev.loc[s,game['TrueSS']].values[0],\
                                             teams.loc[game['TrueSS']].values[0],\
                                   game['ScoreSS'],'X' if game['Result'] == 1 else ' ',game['Prob']))
            print('\t%s\t%s %d\t%s %0.2f' % (seeds_rev.loc[s,game['TrueWS']].values[0],\
                                             teams.loc[game['TrueWS']].values[0],\
                                   game['ScoreWS'],' ' if game['Result'] == 1 else 'X',1-game['Prob']))
            print()
        print('%d of %d games correct --> %d of %d points' % (subset['CorrectPred'].sum(),subset['CorrectPred'].count(),\
                                                             subset['PointsRewarded'].sum(),subset['PointsPossible'].sum()))
        print()
        
        

East: Round 1
	W01	Villanova        87	X 0.94
	W16b	Radford          61	  0.06

	W02	Purdue           74	X 0.89
	W15	CS Fullerton     48	  0.11

	W03	Texas Tech       70	X 0.87
	W14	SF Austin        60	  0.13

	W04	Wichita St       75	  0.53
	W13	Marshall         81	X 0.47

	W05	West Virginia    85	X 0.78
	W12	Murray St        68	  0.22

	W06	Florida          77	X 0.60
	W11a	St Bonaventure   62	  0.40

	W07	Arkansas         62	  0.73
	W10	Butler           79	X 0.27

	W08	Virginia Tech    83	  0.49
	W09	Alabama          86	X 0.51

6 of 8 games correct --> 6 of 8 points

East: Round 2
	W01	Villanova        81	X 0.84
	W09	Alabama          58	  0.16

	W02	Purdue           76	X 0.63
	W10	Butler           73	  0.37

	W03	Texas Tech       69	X 0.37
	W06	Florida          66	  0.63

	W13	Marshall         71	  0.30
	W05	West Virginia    94	X 0.70

1 of 4 games correct --> 2 of 8 points

East: Round 3
	W01	Villanova        90	X 0.72
	W05	West Virginia    78	  0.28

	W02	Purdue           65	  0.34

TeamID
1101    Abilene Chr     
1102    Air Force       
1103    Akron           
1104    Alabama         
1105    Alabama A&M     
1106    Alabama St      
1107    Albany NY       
1108    Alcorn St       
1109    Alliant Intl    
1110    American Univ   
1111    Appalachian St  
1112    Arizona         
1113    Arizona St      
1114    Ark Little Rock 
1115    Ark Pine Bluff  
1116    Arkansas        
1117    Arkansas St     
1118    Armstrong St    
1119    Army            
1120    Auburn          
1121    Augusta         
1122    Austin Peay     
1123    Ball St         
1124    Baylor          
1125    Belmont         
1126    Bethune-Cookman 
1127    Binghamton      
1128    Birmingham So   
1129    Boise St        
1130    Boston College  
              ...       
1437    Villanova       
1438    Virginia        
1439    Virginia Tech   
1440    VMI             
1441    W Carolina      
1442    W Illinois      
1443    WKU             
1444    W Michigan      
1445    W Salem St